In [1]:
import pandas as pd
import unicodedata
import requests
import json
import math
#df = pd.read_csv('../../league_wide/wowy/player_large.csv')
def passing_data(ps = False,update=True):
    url = 'https://api.pbpstats.com/get-totals/nba'
    stype = 'Regular Season'
    trail = ''
    if ps == True:
        stype='Playoffs'
        trail ='/playoffs'
    frames = []
    start_year = 2014
    if update == True:
        df = pd.read_csv('passing.csv')
        df = df[df.year<2024]
        frames.append(df)
        start_year=2024
    for year in range(stary_year,2025):
        #print(str(year-1)+'-'+str(year)[-2:])

        params = {
            "Season": str(year-1)+'-'+str(year)[-2:],
            "SeasonType": stype,
            "Type": 'Player'
        }
        response = requests.get(url, params=params)
        response_json = response.json()
        df = response_json["multi_row_table_data"]
        
        df = pd.DataFrame(df)
        #print(df['Time Of Poss'])
        #print(df.head)

        df2 = pd.read_csv(str(year)+trail+'/player_tracking/passing.csv')
        if year ==2024:
            print(df2.sort_values(by = 'PotentialAST').head(40))
        df2.rename(columns = {'PLAYER':'Name'}, inplace = True)
        df3 =  pd.read_csv(str(year)+trail+'/player_tracking/touches.csv')
        df3.rename(columns = {'Player':'Name'}, inplace = True)
        two = df.merge(df2,on='Name',how ='left',suffixes=('', '_y'))
        #print(two.columns)
        #print(df2.columns)
        #print(df2.columns)
        merged = two.merge(df3,on='Name',how ='left',suffixes=('', '_y'))
        pre = []
        post= []
        for col in merged.columns:
            if "\xa0" in col:
                pre.append(col)
                norm=  unicodedata.normalize('NFKD', col)

                merged.rename(columns = {col:norm}, inplace = True)
        merged = merged.fillna(0)
        merged['Points Unassisted'] = merged['PtsUnassisted2s']+merged['PtsUnassisted3s']
        merged['UAFGM'] = (merged['PtsUnassisted2s']/2)+(merged['PtsUnassisted3s']/3)
        merged['UAPTS'] = merged['Points Unassisted'] 
        merged['on-ball-time'] = merged['Time OfPoss']
        merged['High Value Assist %'] = 100* (merged['ThreePtAssists'] +merged['AtRimAssists'])/merged['Assists']

        merged['on-ball-time%'] = 100* 2* (merged['Time OfPoss'])/(merged['Minutes'])
        merged['TSA'] = (merged['Points']/ (merged['TsPct']*2))
        
        
        merged['Potential Assists'] = merged['PotentialAST']
        merged['Passes'] = merged['PassesMade']
        
        merged['PotAss/Passes'] = merged['Potential Assists']/merged['Passes']
        #merged['Assist PPP'] = merged['Potential Assists']/(75* merged['Assists']/merged['OffPoss'])
        merged['Assist PPP'] = (merged['AST PTSCreated'])/merged['Potential Assists']
        #merged['TOUCHES'] = merged['TOUCHES']/merged['GP']	
        merged['POT_AST_PER_MIN'] = merged['Potential Assists']/(merged['on-ball-time'])
        merged['year'] = year
        #print(*merged['Fg3Pct'])
        #three_p=  (2/(1+math.e**-merged['FG3A'].sum())-1)*(merged['Fg3Pct'].mean()
        #merged['Box Creation'] = merged['Assists']*0.1843+(merged['Points']+merged['Turnovers'])*0.0969-2.3021*(three_p)+0.0582*(merged['Assists'] *(merged['Points']+merged['Turnovers'])*three_p)-1.1942
        frames.append(merged)
        print('Season done ' +str(year))
    df = pd.concat(frames)
    return df

passing = passing_data()
#passing_ps = passing_data(ps=True)
#merged['testas'] = merged['TwoPtAssists']*2+ merged['ThreePtAssists']*3

Season done 2014
Season done 2015
Season done 2016
Season done 2017
Season done 2018
Season done 2019
Season done 2020
Season done 2021
Season done 2022
Season done 2023
                 PLAYER TEAM  GP  W  L   MIN  PassesMade  PassesReceived  AST  \
409        R.J. Hampton  MIA   1  0  1   2.0           2               1    0   
27        Armoni Brooks  BKN   6  3  3  49.0          41              45    1   
297    Keyontae Johnson  OKC   1  0  1   6.0           2               1    0   
158          GG Jackson  MEM   3  0  3   7.0           7               5    0   
438  Shaquille Harrison  MEM   3  1  2   6.0           8               7    0   
279        Kaiser Gates  NOP   1  0  1   7.0          10               8    0   
172          Gui Santos  GSW   1  0  1   8.0           2               3    0   
439        Sidy Cissoko  SAS   1  0  1   4.0           4               3    0   
152       Evan Fournier  NYK   1  1  0  16.0           9              12    1   
174     Harry Giles 

In [2]:
columns = ['EntityId','Name','Points','on-ball-time%','on-ball-time','UAPTS','TSA','OffPoss','Potential Assists','Travels','TsPct',
        'Turnovers','Passes','PassesReceived','PotAss/Passes','UAFGM','High Value Assist %','Assist PPP','TOUCHES','Avg Sec PerTouch',
           'AST PTSCreated','Assists','SecondaryAST','POT_AST_PER_MIN','ThreePtAssists','AtRimAssists','Time OfPoss','ASTAdj','BadPassTurnovers',
       'Avg Drib PerTouch','PtsUnassisted2s','PtsUnassisted3s','Fg3Pct','FG3A','FG3M','OffPoss','GP','Minutes','year']
rs=passing[columns]
#ps=passing_ps[columns]
rs.to_csv('passing.csv',index =False)
#ps.to_csv('../data/passing_ps.csv',index = False)
#print(rs.sort_values(by=)

In [3]:
avg = pd.read_html('https://www.basketball-reference.com/leagues/NBA_stats_per_poss.html')[0]
avg.columns = avg.columns.droplevel()
avg = avg.dropna(subset='Season')
avg = avg[avg.Season!='Season']

avg = avg.dropna()
avg['PTS'] = avg['PTS'].astype(float)
avg['FGA'] = avg['FGA'].astype(float)
avg['FTA'] = avg['FTA'].astype(float)

avg.head(87)

,Rk,Season,Lg,Age,Ht,Wt,G,FG,FGA,3P,...,PTS,FG%,3P%,FT%,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg
0,1,2023-24,NBA,26.5,6-7,217,334,42.0,89.3,12.7,...,114.7,.470,.363,.785,99.3,.541,12.5,25.1,.202,114.7
1,2,2022-23,NBA,26.1,6-6,216,1230,42.0,88.4,12.4,...,114.8,.475,.361,.782,99.2,.545,12.5,24.0,.208,114.8
2,3,2021-22,NBA,26.1,6-6,215,1230,41.1,89.2,12.6,...,112.0,.461,.354,.775,98.2,.532,12.3,23.2,.192,112.0
3,4,2020-21,NBA,26.1,6-6,217,1080,41.3,88.6,12.7,...,112.3,.466,.367,.778,99.2,.538,12.4,22.2,.192,112.3
4,5,2019-20,NBA,26.1,6-6,218,1059,40.4,87.9,12.1,...,110.6,.460,.358,.773,100.3,.529,12.8,22.5,.201,110.6
5,6,2018-19,NBA,26.3,6-6,219,1230,40.8,88.6,11.3,...,110.4,.461,.355,.766,100.0,.524,12.4,22.9,.198,110.4
6,7,2017-18,NBA,26.4,6-7,220,1230,40.5,87.9,10.7,...,108.6,.460,.362,.767,97.3,.521,13.0,22.3,.193,108.6
7,8,2016-17,NBA,26.6,6-7,221,1230,40.2,88.0,9.9,...,108.8,.457,.358,.772,96.4,.514,12.7,23.3,.209,108.8
8,9,2015-16,NBA,26.7,6-7,222,1230,39.6,87.6,8.8,...,106.4,.452,.354,.757,95.8,.502,13.2,23.8,.209,106.4
9,10,2014-15,NBA,26.7,6-7,223,1230,39.6,88.3,8.3,...,105.6,.449,.350,.750,93.9,.496,13.3,25.1,.205,105.6


In [4]:
avg['TS%'] = avg['PTS']/(2*(avg['FGA']+.44*avg['FTA']))
avg

,Rk,Season,Lg,Age,Ht,Wt,G,FG,FGA,3P,...,FG%,3P%,FT%,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,TS%
0,1,2023-24,NBA,26.5,6-7,217,334,42.0,89.3,12.7,...,.470,.363,.785,99.3,.541,12.5,25.1,.202,114.7,0.576846
1,2,2022-23,NBA,26.1,6-6,216,1230,42.0,88.4,12.4,...,.475,.361,.782,99.2,.545,12.5,24.0,.208,114.8,0.581066
2,3,2021-22,NBA,26.1,6-6,215,1230,41.1,89.2,12.6,...,.461,.354,.775,98.2,.532,12.3,23.2,.192,112.0,0.566091
3,4,2020-21,NBA,26.1,6-6,217,1080,41.3,88.6,12.7,...,.466,.367,.778,99.2,.538,12.4,22.2,.192,112.3,0.571583
4,5,2019-20,NBA,26.1,6-6,218,1059,40.4,87.9,12.1,...,.460,.358,.773,100.3,.529,12.8,22.5,.201,110.6,0.564424
5,6,2018-19,NBA,26.3,6-6,219,1230,40.8,88.6,11.3,...,.461,.355,.766,100.0,.524,12.4,22.9,.198,110.4,0.559407
6,7,2017-18,NBA,26.4,6-7,220,1230,40.5,87.9,10.7,...,.460,.362,.767,97.3,.521,13.0,22.3,.193,108.6,0.556216
7,8,2016-17,NBA,26.6,6-7,221,1230,40.2,88.0,9.9,...,.457,.358,.772,96.4,.514,12.7,23.3,.209,108.8,0.552441
8,9,2015-16,NBA,26.7,6-7,222,1230,39.6,87.6,8.8,...,.452,.354,.757,95.8,.502,13.2,23.8,.209,106.4,0.541487
9,10,2014-15,NBA,26.7,6-7,223,1230,39.6,88.3,8.3,...,.449,.350,.750,93.9,.496,13.3,25.1,.205,105.6,0.533851


In [5]:
avg.to_csv('avg_shooting.csv',index = False)
avg = avg[['Season','ORtg']]
avg.to_csv('team_avg.csv',index = False)